In [ ]:
# Developer: Angel Chiu / @9ukei
# Program Name: air_ticket_scraping.ipynb
# Date: 2023/08/14
# Function: 1. Scraping Air Ticket Prices
#           2. Saving Air Ticket Price Data as CSV
#           3. Uploading Air Ticket Price Data to Google Sheets
#           4. Regular Notifications via Line Notify: Top 3 Airlines with Lowest Prices

## Prerequisites
1. First, ensure that the `selenium`, `BeautifulSoup4`, and `lxml` libraries are installed on your device.
2. If you have already installed these required packages for your project, you can skip this step.

In [ ]:
# install selenium, BeautifulSoup4, lxml
# !pip install selenium
# !pip install BeautifulSoup4
# !pip install lxml

## Web Scraping

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import datetime
import pandas as pd

In [2]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

# Crawl the URL that you wanted to scrape on Line travel
# Here, I will take the example of comparing flight prices for flights from Taipei (TPE) to Osaka (OSA) for the dates 9/11 to 9/16.
air_ticket_url = '''
https://travel.line.me/flights/list?roundType=1&cabinClass=1&numOfAdult=2&numOfChildren=0&numOfBaby=0&linePointsRebateOnly=1&departureAirports=&departureCities=TPE&departureDates=1694390400000&arrivalAirports=&arrivalCities=OSA&departureAirports=&departureCities=OSA&departureDates=1694822400000&arrivalAirports=&arrivalCities=TPE
'''

# This number of seconds can be adjusted and increased according to the network delay problem. 
# It is recommended to stay at least 10 seconds or more.
driver.implicitly_wait(10)
driver.get(air_ticket_url)

# load more results to maximize the scraping
def page_scrolldown():
    try:
        for i in range(1,20):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(0.7)
    except:
        print('Check to see if any code is causing the error.')
        pass
page_scrolldown()

# view the html source code of the website
driver.page_source

html_source_code = driver.page_source

driver.close()

/var/folders/0t/tf8z0v6j5cq6506t24xz7j5w0000gn/T/ipykernel_5278/3714825685.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/hsin/chromedriver-mac-arm64/chromedriver')


In [ ]:
# How to strip comma in Python string
# https://stackoverflow.com/questions/16233593/how-to-strip-comma-in-python-string
#a = total_price.replace(',', '')
#print(int(a))

In [3]:
soup = BeautifulSoup(html_source_code, 'html.parser')

# check source code without tags
# print(soup.text)

# Display the total number of tickets found
record = soup.select_one('#__next > div.css-1cp3u8n.e1ugfpty0 > div:nth-child(2) > span')
print('搜尋的機票比價總筆數：',record.text)
#

time_loc_list = []

for time_loc_info in soup.select('#__next > div.css-1cp3u8n.e1ugfpty0 > div:nth-child(2)'):
    for tl in time_loc_info.select('.css-nkthol.ejxn77z1, .css-1qzlsgj.e1fe20ih2'):
        time = tl.text
        time_loc_list.append(time)

# 遍歷 time_loc_list 中的元素，每次取八個元素組合成一個元組，並將這些元組放入 tuple_list 中
tuple_list = [tuple(time_loc_list[i:i+8]) for i in range(0, len(time_loc_list), 8)]
# print(tuple_list)

df_time = pd.DataFrame(tuple_list, columns=["起飛時間(出發)","起飛地點(出發)","抵達時間(出發)","抵達地點(出發)", "起飛時間(回程)","起飛地點(回程)", "抵達時間(回程)","抵達地點(回程)"])

result = []

# Depart/Arrived time + location (aboard/arrived)
for time_loc_info in soup.select('#__next > div.css-1cp3u8n.e1ugfpty0 > div:nth-child(2)'):
    for tl in time_loc_info.select('.css-1eowobi .css-j7qwjs'):
        time_loc = tl.text
        # print(time_loc)

air_ticket_info = soup.find_all(class_='css-1eowobi')

for ticket_info in air_ticket_info:
    # airline company
    airline = ticket_info.find(class_='css-84a4s3 e1fe20ih3').getText().strip()
    # air ticket provider
    ticket_site = ticket_info.find(class_='css-6x2xcr e1fe20ih2').getText().strip()   
    # ticket per price
    ticket_per_price = ticket_info.find(class_='css-iw7h7v ejxn77z0').getText().strip().replace(',', '')
    # total ticket price(2 ppl)
    total_price = ticket_info.find(class_='css-wycfi3 e1fe20ih3').getText().strip('')[9:].replace(',', '')
    # ticket purchase url
    ticket_purchase_url = 'https://travel.line.me/' + ticket_info.find('a').get('href')

    result.append((airline,ticket_site,int(ticket_per_price),int(total_price), ticket_purchase_url))

    #print(airline,ticket_site, ticket_per_price, total_price)

df = pd.DataFrame(result, columns=["航空公司", "購買網站", "一人價格(TWD)", "兩人總價(TWD)", "買票去！"])

#print(df.head(30))

# print(result) 

搜尋的機票比價總筆數： 687 個航班


In [ ]:
# df_time

In [ ]:
# df

In [4]:
# 如果 df1 和 df2 的列數相同，但是欄位名稱不同，您可以使用 concat 函數將它們合併在一起。concat 函數可以在指定的軸上連接多個 DataFrame。
# 在您的情況下，您可以選擇在列軸上進行連接。

ticket_full_info = pd.concat([df_time, df], axis=1)
ticket_full_info.align
ticket_full_info.index +=1
print(ticket_full_info.head(10))

   起飛時間(出發) 起飛地點(出發) 抵達時間(出發) 抵達地點(出發) 起飛時間(回程) 起飛地點(回程) 抵達時間(回程) 抵達地點(回程)  \
1     21:55      TPE    01:30      ICN    02:30      ICN    04:05      TPE   
2     21:55      TPE    01:30      ICN    07:50      ICN    09:25      TPE   
3     18:00      TPE    21:35      ICN    02:30      ICN    04:05      TPE   
4     13:50      TSA    17:25      GMP    07:50      ICN    09:25      TPE   
5     13:50      TSA    17:25      GMP    02:30      ICN    04:05      TPE   
6     18:00      TPE    21:35      ICN    07:50      ICN    09:25      TPE   
7     21:55      TPE    01:30      ICN    09:05      ICN    10:50      TPE   
8     10:35      TPE    14:00      ICN    07:50      ICN    09:25      TPE   
9     13:50      TSA    17:25      GMP    09:05      ICN    10:50      TPE   
10    21:55      TPE    01:30      ICN    12:55      ICN    17:40      TPE   

           航空公司       購買網站  一人價格(TWD)  兩人總價(TWD)  \
1          台灣虎航  ezfly 易飛網       5468      10936   
2      台灣虎航、真航空  BudgetAir       6953

In [5]:
now = datetime.datetime.now()
# 若月份非為 10~12 月，則在 1~9 月份後面加一個 0
if now.month <= 10:
    date = str(now.year)+'0'+str(now.month)+str(now.day)
else:
    pass

# 現在時間
loc_dt = datetime.datetime.today() 
loc_dt_format = loc_dt.strftime("%Y/%m/%d %H:%M:%S")

# dataframe to csv
scapring_date = date
ticket_full_info.to_csv(f"{scapring_date}_air_ticket_full_info.csv")

## Sync data to Google Sheets by Pygsheets
- To sync the air ticket data to `Google Sheets` in real time, we can use a Python module called `Pygsheets` to control Google Sheet API. 

In [ ]:
# !pip install pygsheets 

In [6]:
import pygsheets

auth_file = "credentials/client_secret.json"
gc = pygsheets.authorize(service_file = auth_file)

# sheet read by pygsheets
sheet_url = "https://docs.google.com/spreadsheets/d/1p5wOveW2B_bbbaBQdvpYdWcbFm_5XrRuec6Mhctqlx4/" 
sheet = gc.open_by_url(sheet_url)

#選取by名稱
air_ticket_sheet_01 = sheet.worksheet_by_title("air ticket price comparison")

# 更新工作表中的值
# title_date = 'A1'
# air_ticket_sheet_01.update_values(title_date, [['9/4 ~ 9/9 韓國台灣來回機票即時比價報表' + '\n' + loc_dt_format]])
attributes = 'A1'
air_ticket_sheet_01.update_values(attributes, [["起飛時間(出發)","起飛地點(出發)","抵達時間(出發)","抵達地點(出發)", "起飛時間(回程)","起飛地點(回程)", "抵達時間(回程)","抵達地點(回程)","航空公司", "購買網站", "一人價格(TWD)", "兩人總價(TWD)", "買票去！"]])
start_record = 'A2'
# `df.values.tolist()` method can transform the data type `dataframe` to `list`
air_ticket_sheet_01.update_values(start_record, ticket_full_info.values.tolist())


## Line Notify 串接 + Cron Job

In [ ]:
# !pip3 install schedule

In [ ]:
import requests
import schedule
import time

def send_notification():

    # Read the data from Google Sheets
    data = air_ticket_sheet_01.get_all_records()

    # Extract the top three combinations of airlines with the lowest prices
    top_three_rows = data[:3]
    
    # Send LINE Notify notification   
    line_notify_url = "https://notify-api.line.me/api/notify"

    msg = '\n9/4 ~ 9/9 台灣韓國來回機票\n即時比價報表\n\n'
    for row in top_three_rows:
        msg += f'''💎航空公司: {row["航空公司"]}\n💎一人價格(TWD): {row["一人價格(TWD)"]}元\n💎兩人總價(TWD): {row["兩人總價(TWD)"]}元\n✈️哪次不衝了，買票去！\n{row["買票去！"]}\n\n'''
    
    # for test
    # payload={'message':{msg}}
    payload={'message':{msg}}
    headers = {'Authorization': 'Bearer ' + '4APR0M6N9eQXzpCUC1SzuHLu7uAyMZN4cN4U71Q52ct'}
    # Status code
    response = requests.request("POST", line_notify_url, headers=headers, data=payload)
    print(response.text)

# Set the time for sending notifications (Example: Every day at 10 AM)
schedule.every().day.at("3:25").do(send_notification)
# schedule.every(60).seconds.do(send_notification)

# Infinite loop to keep the script running
while True:
    schedule.run_pending()
    time.sleep(1)